In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import emcee

/usr/local/lib/python2.7/dist-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
import astropy.units as u

In [3]:
from grbeams.distributions import *
from grbeams.scenarios import *

In [4]:
rateunits = u.megaparsec**-3*u.year**-1
data = np.loadtxt('../O1/bns_rate_posterior.txt')
rate, pdf = data[:,0]*rateunits, data[:,1]*rateunits
bns_prior = BNSDistribution(rate, pdf)

In [5]:
o1scenario = Scenario(bns_prior)

In [6]:
grb_rate = 3.0 * rateunits

In [7]:
class BeamingAnglePosterior(Distribution):

    def __init__(self, 
                 observing_scenario, 
                 efficiency_prior=DeltaDistribution(1.0),#'delta,1.0',
                 grb_rate=3 / u.gigaparsec**3 / u.year):
        """
        The posterior probability distribution on the beaming angle, theta.
        
        Parameters
        ----------
        observing_scenario : Scenario object
            The observing scenario, e.g. O1.
        efficiency_prior : str
            The shape of the prior on epsilon, the efficiency at which
            BNS mergers produce sGRBs.
        grb_rate : 
            The rate of GRBs in the galaxy.
        """

        self.efficiency_prior = efficiency_prior

        # --- Prior Setup
        
        self.theta_range = np.arange(0.0, 90.0, 0.01)
        self.efficiency_range = efficiency_prior.range
        
        # --- Astro configuration
        self.grb_rate = grb_rate
        self.scenario = observing_scenario

    def get_theta_pdf_kde(self, bandwidth=1.0):

        self.theta_grid  = self.theta_range #np.arange(self.theta_range[0], self.theta_range[1], 0.01)
        self.theta_pdf_kde  = kde_sklearn(x=self.theta_samples,
                                          x_grid=self.theta_grid, 
                                            bandwidth=bandwidth, 
                                            algorithm='kd_tree') 
        self.theta_bounds, self.theta_median, self.theta_posmax = \
                characterise_dist(self.theta_grid, self.theta_pdf_kde, 0.95)

    def sample_theta_posterior(self, nburnin=100, nsamp=500, nwalkers=100):
        """
        Use emcee ensemble sampler to draw samples from the ndim parameter space
        comprised of (theta, efficiency, delta_theta, ...) etc

        The dimensionality is determined in __init__ based on the priors used

        The probability function used is the comp_theta_prob() method
        """

        theta0 = (max(self.theta_range)-min(self.theta_range)) * np.random.rand(nwalkers)
        p0 = theta0.reshape((nwalkers, self.efficiency_prior.ndim))
        
        if self.efficiency_prior.ndim==2:
            
            efficiency0 = (max(self.efficiency_prior.range)-min(self.efficiency_prior.range)) *\
                    np.random.rand(nwalkers)
            
            p0 = np.transpose(np.array([theta0, efficiency0]))
            
        # Inititalize sampler
        if self.efficiency_prior.name=='delta':
            self.sampler = emcee.EnsembleSampler(nwalkers, self.efficiency_prior.ndim,
                    self.comp_theta_prob, args=[self.efficiency_prior.x])
        else:
            self.sampler = emcee.EnsembleSampler(nwalkers, self.efficiency_prior.ndim, 
                                                 self.comp_theta_prob_nparam)

        # Burn-in
        pos, prob, state = self.sampler.run_mcmc(p0, nburnin)
        self.sampler.reset()

        # Draw samples
        self.sampler.run_mcmc(pos, nsamp)

        # 1D array with samples for convenience
        if self.efficiency_prior.ndim==1:
            self.theta_samples = np.concatenate(self.sampler.flatchain)
        else:
            self.theta_samples = self.sampler.flatchain[:,0]
            self.efficiency_samples = self.sampler.flatchain[:,1]


        # Create bppu posterior instance for easy conf intervals and
        # characterisation
        #self.theta_pos = bppu.PosteriorOneDPDF('theta', self.theta_samples,
        #        injected_value=self.sim_theta)
        
        return self.sampler.flatchain

    def comp_theta_prob_nparam(self, x, fixed_args=None):
        return self.logpdf(theta=x[0], efficiency=x[1])

            
    def logpdf(self,theta,efficiency):
        """
        Perform the rate->theta posterior transformation.

        Here's the procedure:
        1) Given an efficiency and theta angle, find the corresponding cbc rate
        according to Rcbc = Rgrb / (1-cos(theta))
        2) evaluate rate posterior at this value of the cbc rate
        3) The theta angle posterior is then just jacobian * rate
        posterior[rate=rate(theta)]
        """
        #print efficiency, self.comp_efficiency_prob(efficiency)
        if (theta>=min(self.theta_range)) and \
                (theta<max(self.theta_range)):

            # Get BNS rate from theta, efficiency
            bns_rate = self.scenario.cbc_rate_from_theta(self.grb_rate, theta, efficiency)

            # Get value of rate posterior at this rate
            bns_rate_pdf = self.scenario.comp_bns_rate_pdf(bns_rate)

            # Compute jacobian
            jacobian = self.compute_jacobian(efficiency,theta).value

            theta_prob = bns_rate_pdf + np.log(jacobian) \
                    + np.log(self.comp_efficiency_prob(efficiency))


        else:
            # outside of prior ranges
            theta_prob = -np.inf

        return theta_prob
    
    def compute_jacobian(self,efficiency,theta):
        """
        Compute the Jacboian for the transformation from rate to angle
        """

        denom=efficiency*(np.cos(theta * np.pi/180)-1)
        return abs(2.0*self.grb_rate * np.sin(theta * np.pi / 180.0) /
                (denom*denom) )


    def comp_efficiency_prob(self,efficiency):
        """
        Prior on the BNS->GRB efficiency
        """
        return self.efficiency_prior.pdf(efficiency)

Construct the theta posterior distribution for the jets.

In [16]:
o1scenario.cbc_rate_from_theta(theta = 30, grb_rate=grb_rate, efficiency=0.3)

<Quantity -74.64101615137758 1 / (Mpc3 yr)>

In [9]:
thetapos = BeamingAnglePosterior(o1scenario, DeltaDistribution(0.3), grb_rate=grb_rate )

In [11]:
np.exp(thetapos.logpdf(0.3, 0.3))

1857688383.3334398

In [14]:
thetapos.logpdf(theta = 30, efficiency = 0.3)

7.5267680522895066

In [15]:
thetapos.sample_theta_posterior()

emcee: Exception while calling your likelihood function:
  params: [ 11.91949427]
  args: [0.3]
  kwargs: {}
  exception:


Traceback (most recent call last):
  File "/home/daniel/.virtualenvs/grbeams/lib/python2.7/site-packages/emcee/ensemble.py", line 505, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "<ipython-input-13-07d19ebf4944>", line 116, in comp_theta_prob
    bns_rate_pdf = self.scenario.comp_bns_rate_pdf(bns_rate)
  File "/home/daniel/repositories/grbeams/grbeams/scenarios.py", line 68, in comp_bns_rate_pdf
    return self.bns_prior.pdf(rate)
  File "/home/daniel/repositories/grbeams/grbeams/scenarios.py", line 36, in pdf
    if rate > max(self.rates): return 0
  File "/home/daniel/.virtualenvs/grbeams/lib/python2.7/site-packages/astropy/units/quantity.py", line 409, in __array_prepare__
    result = self._new_view(obj, result_unit)
  File "/home/daniel/.virtualenvs/grbeams/lib/python2.7/site-packages/astropy/units/quantity.py", line 589, in _new_view
    view.__array_finalize__(self)
  File "/home/daniel/.virtualenvs/grbeams/lib/python2.7/site-packages/astropy/units/quantit

KeyboardInterrupt: 